# 前処理

In [28]:
import os
import json

import numpy as np
import pandas as pd

from sklearn.model_selection import KFold
from pycaret.classification import *

In [29]:
from eeg_utils import *

_,train_target = prepare_all()

In [30]:
# 各cvの結果を一つのDataFrameにする
def load_vals(sub,isplit, data_type):
    vals={}
    for model in "conv1d conv2d lstm r3d mc3 r2plus1d".split():
        filename = f"{model}_{data_type}/{sub}_{isplit}.json"
        key=f"{sub}-{isplit}"
        with open(filename) as f:
            d = json.load(f)[key]

        vals[model]=d

    df=pd.DataFrame()
    for k,v in vals.items():
        df[k]=np.argmax(v,axis=1)
    if data_type=="val":
        df["target"]=train_target[f"{sub}"][isplit]%10-1
    return df

In [31]:
def calc_accuracy(df):
    accuracy={}
    for model in df.columns:
        if model!="target":
            acc = sum(df[model]==df["target"])/len(df)
            accuracy[model]=acc
    return accuracy

# 学習

In [32]:
# predictions = {}
save_path = "stacking_model3"
os.makedirs(save_path,exist_ok=True)
for sub in range(5):
    sub_str = f"{sub:04d}"
    for isplit in range(3):
        df_val = load_vals(sub_str,isplit,"val")
        df_test = load_vals(sub_str,isplit,"test")
        setup(data=df_val,categorical_features=list(df_val.columns)[:-1], target="target", fold=10, session_id=1, ignore_features=["isplit"])

        best_model = compare_models()
        results=pull()
        save_model(best_model, f"{save_path}/{sub_str}-{isplit}")

,Description,Value
0,Session id,1
1,Target,target
2,Target type,Multiclass
3,Original data shape,"(158, 7)"
4,Transformed data shape,"(158, 17)"
5,Transformed train set shape,"(110, 17)"
6,Transformed test set shape,"(48, 17)"
7,Ignore features,1
8,Categorical features,6
9,Preprocess,True


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lr,Logistic Regression,0.9818,0.0000,0.9818,0.9864,0.9801,0.9690,0.9715,0.1520
svm,SVM - Linear Kernel,0.9818,0.0000,0.9818,0.9856,0.9805,0.9700,0.9725,0.0190
rf,Random Forest Classifier,0.9727,0.9984,0.9727,0.9803,0.9716,0.9553,0.9589,0.0540
gbc,Gradient Boosting Classifier,0.9636,0.0000,0.9636,0.9735,0.9627,0.9407,0.9454,0.0500
et,Extra Trees Classifier,0.9636,0.9979,0.9636,0.9735,0.9623,0.9412,0.9459,0.0390
lightgbm,Light Gradient Boosting Machine,0.9636,0.9957,0.9636,0.9682,0.9625,0.9398,0.9422,0.0730
ridge,Ridge Classifier,0.9545,0.0000,0.9545,0.9652,0.9529,0.9272,0.9330,0.0230
lda,Linear Discriminant Analysis,0.9545,0.0000,0.9545,0.9652,0.9529,0.9272,0.9330,0.0160
knn,K Neighbors Classifier,0.9455,0.9934,0.9455,0.9545,0.9438,0.9103,0.9151,0.0170
nb,Naive Bayes,0.9455,0.9876,0.9455,0.9576,0.9427,0.9113,0.9185,0.0160


Transformation Pipeline and Model Successfully Saved


,Description,Value
0,Session id,1
1,Target,target
2,Target type,Multiclass
3,Original data shape,"(160, 7)"
4,Transformed data shape,"(160, 19)"
5,Transformed train set shape,"(112, 19)"
6,Transformed test set shape,"(48, 19)"
7,Ignore features,1
8,Categorical features,6
9,Preprocess,True


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
knn,K Neighbors Classifier,0.9364,0.9614,0.9364,0.9452,0.9358,0.8961,0.9007,0.0180
lr,Logistic Regression,0.8833,0.0000,0.8833,0.9002,0.8766,0.8107,0.8217,0.0180
lightgbm,Light Gradient Boosting Machine,0.8833,0.9622,0.8833,0.9024,0.8771,0.8107,0.8221,0.0760
lda,Linear Discriminant Analysis,0.8742,0.0000,0.8742,0.8996,0.8706,0.7981,0.8093,0.0160
rf,Random Forest Classifier,0.8735,0.9489,0.8735,0.8897,0.8708,0.7972,0.8066,0.0470
nb,Naive Bayes,0.8652,0.9380,0.8652,0.8926,0.8649,0.7836,0.7943,0.0180
ridge,Ridge Classifier,0.8652,0.0000,0.8652,0.8918,0.8582,0.7817,0.7950,0.0170
gbc,Gradient Boosting Classifier,0.8644,0.0000,0.8644,0.9036,0.8574,0.7807,0.7996,0.0550
et,Extra Trees Classifier,0.8561,0.9374,0.8561,0.8859,0.8536,0.7696,0.7828,0.0420
dt,Decision Tree Classifier,0.8553,0.8720,0.8553,0.8816,0.8533,0.7654,0.7804,0.0150


...

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lr,Logistic Regression,0.9545,0.0000,0.9545,0.9667,0.9534,0.9235,0.9311,0.0210
rf,Random Forest Classifier,0.9455,0.9956,0.9455,0.9589,0.9442,0.9070,0.9160,0.0410
svm,SVM - Linear Kernel,0.9364,0.0000,0.9364,0.9513,0.9352,0.8948,0.9018,0.0200
ridge,Ridge Classifier,0.9288,0.0000,0.9288,0.9441,0.9274,0.8853,0.8936,0.0180
lda,Linear Discriminant Analysis,0.9288,0.0000,0.9288,0.9441,0.9274,0.8853,0.8936,0.0140
et,Extra Trees Classifier,0.9280,0.9961,0.9280,0.9413,0.9273,0.8800,0.8888,0.0380
lightgbm,Light Gradient Boosting Machine,0.9280,0.9926,0.9280,0.9415,0.9269,0.8820,0.8906,0.0670
knn,K Neighbors Classifier,0.9273,0.9827,0.9273,0.9460,0.9258,0.8778,0.8891,0.0200
dt,Decision Tree Classifier,0.9189,0.9314,0.9189,0.9399,0.9145,0.8652,0.8795,0.0200
gbc,Gradient Boosting Classifier,0.9189,0.0000,0.9189,0.9339,0.9178,0.8670,0.8769,0.0520


Transformation Pipeline and Model Successfully Saved


,Description,Value
0,Session id,1
1,Target,target
2,Target type,Multiclass
3,Original data shape,"(160, 7)"
4,Transformed data shape,"(160, 17)"
5,Transformed train set shape,"(112, 17)"
6,Transformed test set shape,"(48, 17)"
7,Ignore features,1
8,Categorical features,6
9,Preprocess,True


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
knn,K Neighbors Classifier,0.8492,0.9289,0.8492,0.8587,0.8355,0.7534,0.7762,0.0190
lr,Logistic Regression,0.8311,0.0000,0.8311,0.8695,0.8237,0.7244,0.7459,0.0190
rf,Random Forest Classifier,0.8227,0.9431,0.8227,0.8441,0.8086,0.7168,0.7419,0.0530
ridge,Ridge Classifier,0.8220,0.0000,0.8220,0.8673,0.8147,0.7115,0.7358,0.0180
nb,Naive Bayes,0.8129,0.9247,0.8129,0.8576,0.8052,0.7030,0.7271,0.0190
lda,Linear Discriminant Analysis,0.8129,0.0000,0.8129,0.8509,0.8044,0.6939,0.7168,0.0140
ada,Ada Boost Classifier,0.8121,0.0000,0.8121,0.8370,0.7980,0.6883,0.7129,0.0290
svm,SVM - Linear Kernel,0.8038,0.0000,0.8038,0.8230,0.7986,0.6815,0.6941,0.0190
lightgbm,Light Gradient Boosting Machine,0.8038,0.9368,0.8038,0.8526,0.7981,0.6859,0.7114,0.0670
gbc,Gradient Boosting Classifier,0.7871,0.0000,0.7871,0.8438,0.7829,0.6641,0.6885,0.0510


Transformation Pipeline and Model Successfully Saved


# 予測

In [33]:
save_path = "stacking_model3"
predictions={}
for sub in range(5):
    sub_str = f"{sub:04d}"
    predictions[sub_str]=[]

    for isplit in range(3):
        df_test = load_vals(sub_str,isplit,"test")

        filename=os.path.join(save_path,f"{sub_str}-{isplit}")
        best_model = load_model(filename)
        pred = predict_model(best_model, data=df_test)
        predictions[sub_str].append(pred)


Transformation Pipeline and Model Successfully Loaded


Transformation Pipeline and Model Successfully Loaded


Transformation Pipeline and Model Successfully Loaded


Transformation Pipeline and Model Successfully Loaded


Transformation Pipeline and Model Successfully Loaded


...

Transformation Pipeline and Model Successfully Loaded


Transformation Pipeline and Model Successfully Loaded


Transformation Pipeline and Model Successfully Loaded


Transformation Pipeline and Model Successfully Loaded


Transformation Pipeline and Model Successfully Loaded


In [34]:
df_all_pred=[]
for sub in "0000 0001 0002 0003 0004".split():
    df_pred = pd.concat([predictions[sub][i]["prediction_label"] for i in range(3)],axis=1)
    df_pred["sub"]=sub
    df_all_pred.append(df_pred)
df_all_pred = pd.concat(df_all_pred)
df_all_pred

,prediction_label,prediction_label,prediction_label,sub
0,2,2,2,0000
1,2,2,2,0000
2,2,2,2,0000
3,2,2,2,0000
4,0,0,0,0000
...,...,...,...,...
154,2,2,2,0004
155,2,2,2,0004
156,1,1,1,0004
157,0,0,1,0004


### 3cvの結果を多数決する。同数の場合は2とする。

In [35]:
df_all_pred["title"]=df_all_pred.apply(lambda row: f"subject{int(row['sub'])}_{row.name:03d}", axis=1)
df_all_pred[["mode0","mode1","mode2"]]=df_all_pred.iloc[:,:3].mode(axis=1)
df_all_pred["final"]=df_all_pred["mode0"]
df_all_pred.loc[df_all_pred['mode2'].notna(), 'final'] = df_all_pred.loc[df_all_pred['mode2'].notna(), 'mode2']
df_all_pred.final = df_all_pred.final.astype(int)
df_all_pred["event"] = df_all_pred.apply(lambda row: event2str3[row["final"]+1], axis=1)
df_all_pred


,prediction_label,prediction_label,prediction_label,sub,title,mode0,mode1,mode2,final,event
0,2,2,2,0000,subject0_000,2.0,NaN,NaN,2,pumping
1,2,2,2,0000,subject0_001,2.0,NaN,NaN,2,pumping
2,2,2,2,0000,subject0_002,2.0,NaN,NaN,2,pumping
3,2,2,2,0000,subject0_003,2.0,NaN,NaN,2,pumping
4,0,0,0,0000,subject0_004,0.0,NaN,NaN,0,frontside_kickturn
...,...,...,...,...,...,...,...,...,...,...
154,2,2,2,0004,subject4_154,2.0,NaN,NaN,2,pumping
155,2,2,2,0004,subject4_155,2.0,NaN,NaN,2,pumping
156,1,1,1,0004,subject4_156,1.0,NaN,NaN,1,backside_kickturn
157,0,0,1,0004,subject4_157,0.0,NaN,NaN,0,frontside_kickturn


In [36]:
df_all_pred[["title","event"]].to_csv("classification1_1028.csv",index=False,header=False)